In [ ]:
import numpy as np
import pandas as pd
from scipy.io import *
from pylab import *
from scipy.signal import *
from pandas import DataFrame as df

In [ ]:
#pfc
dataframe_pfc_conrol=pd.read_csv('PFC_Control.csv', sep=',')
dataframe_pfc_conrol=dataframe_pfc_conrol.drop(columns ='Unnamed: 0')
dataframe_pfc_mutation=pd.read_csv('PFC_Mutation.csv', sep=',')
dataframe_pfc_mutation=dataframe_pfc_mutation.drop(columns ='Unnamed: 0')
df_pfc=dataframe_pfc_mutation.merge(dataframe_pfc_conrol, how='outer')
#hpc
dataframe_hpc_conrol=pd.read_csv('HPC_Control.csv', sep=',')
dataframe_hpc_conrol=dataframe_hpc_conrol.drop(columns ='Unnamed: 0')
dataframe_hpc_mutation=pd.read_csv('HPC_Mutation.csv', sep=',')
dataframe_hpc_mutation=dataframe_hpc_mutation.drop(columns ='Unnamed: 0')
df_hpc=dataframe_hpc_mutation.merge(dataframe_hpc_conrol, how='outer')
#Mutation
df_mutation=dataframe_pfc_mutation.merge(dataframe_hpc_mutation, how='outer')

#control
df_control=dataframe_pfc_conrol.merge(dataframe_hpc_conrol, how='outer')


In [ ]:
def average(df_r, m, nperseg=100, scale=10, step=2):
    length =np.inf
    dataframe = df(columns = ['Rat', 'Condition', 'Epoch','MC','Region','Spectral Analysis Power', 'Spectral Analysis Frequency', 'REM_Data'])
    dataframe['Spectral Analysis Power'] = dataframe['Spectral Analysis Power'].astype(object)
    dataframe['Spectral Analysis Frequency'] = dataframe['Spectral Analysis Frequency'].astype(object)
    dataframe['REM_Data'] = dataframe['REM_Data'].astype(object)
    print(set(df_r.Rat.values))
    for rats in set(df_r.Rat.values):
        if rats !=5:
            print('Rat',rats)
            data =df_r.loc[(df_r.Rat==rats), 'REM Data']
            mc = df_r.loc[(df_r.Rat==rats), 'MC'].values[0]
            region = df_r.loc[(df_r.Rat==rats), 'Region'].values[0]
            length_data = int(np.array([len([float(num) for num in d.split(',')[1:-1]]) for d in data]).mean())
            epochd = np.zeros([length_data])
            e=1
            for d in data:
                d = [float(num) for num in d.split(',')[1:-1]]
                #expell small rem cycles
                if len(d) >= (length_data):
                    lfp=np.array(d)
                    #spectral plot decibel welch
                    x = lfp
#                     print(x)
                    N = x.shape[0]
                    t = np.array(range(len(x)))
                    dt = t[1]-t[0]
                    T = t[-1]
                    x = x - x.mean()
                    f,pxx = welch(x, step, nperseg=N//nperseg)    # Compute Power spectral density Welch
                    Sxx = 2 * dt ** 2 / T * (pxx * conj(pxx)) # Compute spectrum
#                     ylabel('Power [dB]')
#                     xlabel('Frequency[Hz]')
#                     plot(f*1000, (scale*log10(Sxx / max(Sxx))))
        #             xlim([0,200])
        #             savefig('Averaged/%s_Rat%i_Averaged_2_100_20' %(region,rats))
#                     show()
                    #color plot
        #             Fs = 1 / dt               # Define the sampling frequency,
        #             interval = int(N/50)        # ... the interval size,
        #             overlap = int(interval*0.5)  # ... and the overlap intervals

        #                                       Compute the spectrogram
        #             f, t, Sxx = spectrogram(
        #                 x,                  # Provide the signal,
        #                 fs=Fs,                # ... the sampling frequency,
        #                 nperseg=interval,     # ... the length of a segment,
        #                 noverlap=overlap)     # ... the number of samples to overlap,
        #             pcolormesh(t, f*1000, 10 * log10(Sxx),
        #                            cmap='jet')# Plot the result
        #             colorbar()                # ... with a color bar,
        #             xlabel('Time [s]')        # ... and label the axes
        #             ylabel('Frequency [Hz]')
        #             savefig('Averaged/%s_Rat%i_Averaged_Spectrogram_10', %(region,rats))
            #         show()
                    dataframe  = dataframe.append({'Rat': rats, 'Condition': 'Overlapping','Epoch':e, 'MC':mc,'Region':region,\
                                                      'Spectral Analysis Power': (10*log10(Sxx / max(Sxx))).tolist(), 'Spectral Analysis Frequency':(f*1000).tolist(), 'REM_Data':x.tolist()}, ignore_index=True)
                    e+=1

    return dataframe
        

In [ ]:
# dataframe_hpc_control = average(dataframe_hpc_conrol, 100, scale=10)
# dataframe_hpc_control.to_csv('HPC_control_100_2_10.csv', sep = ',',index=False)

# dataframe_pfc_control=average(dataframe_pfc_conrol,100,scale=10)
# dataframe_pfc_control.to_csv('PFC_control_100_2_10.csv', sep = ',',index=False)

# dataframe_hpc_mutation = average(dataframe_hpc_mutation, 100, scale=10)
# dataframe_hpc_mutation.to_csv('HPC_mutation_100_2_10.csv', sep = ',',index=False)
# dataframe_pfc_mutation=average(dataframe_pfc_mutation,100,scale=10)
# dataframe_pfc_mutation.to_csv('PFC_mutation_100_2_10.csv', sep = ',',index=False)
dataframe_hpc = average(df_hpc, 100,scale=10)
dataframe_pfc = average(df_pfc, 100,scale=10)


In [ ]:
dataframe = pd.concat([dataframe_hpc, dataframe_pfc],ignore_index=True, sort=False)

In [ ]:
dataframe.to_csv('Merged_100_2_10.csv', sep = ',',index=False)